<a href="https://colab.research.google.com/github/WadeShadow/nlp2021/blob/main/Lab2/Lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import numpy as np, pandas as pd, tensorflow as tf, requests as rqst, io
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras import layers

rnd = np.random.randint

folder = '/content/drive/MyDrive/Colab Notebooks/NLP_labs/'

In [12]:
file = open(folder+'sentences.txt')
sentences = file.read().split('. ')[:-1]
sentences

['Training a virtual agent to outperform human players can teach us how to optimize different processes in a variety of different and exciting subfields',
 'This is what Google DeepMind did with its popular AlphaGo, which beat the strongest Go player in history and scored a goal that was considered impossible at the time.In this article, we will develop an AI agent that is able to learn how to play the popular game Snake from scratch',
 'To do it, we implement a Deep Reinforcement Learning algorithm using both Keras on top of Tensorflow and PyTorch (both versions are available, you can choose the one you prefer)',
 'This approach consists in the interaction between two components: an environment (the game itself) and an agent (Snake)',
 'The agent collects information about its current state (we will see later what this means) and performs an action accordingly',
 'The environment rewards or punishes the agent based on the performed action',
 'Over time, the agent learns what actions m

### Підібрати наукові статті за обраним напрямком англійською мовою, з яких вилучити 100 речень, які найбільш відповідають зазначеному напрямку

In [22]:
sent_df = []
for sent in sentences:
  sent_df.append({'Sentence':sent, 'Label':'Reinforcement Learning'})

sent_df

[{'Label': 'Reinforcement Learning',
  'Sentence': 'Training a virtual agent to outperform human players can teach us how to optimize different processes in a variety of different and exciting subfields'},
 {'Label': 'Reinforcement Learning',
  'Sentence': 'This is what Google DeepMind did with its popular AlphaGo, which beat the strongest Go player in history and scored a goal that was considered impossible at the time.In this article, we will develop an AI agent that is able to learn how to play the popular game Snake from scratch'},
 {'Label': 'Reinforcement Learning',
  'Sentence': 'To do it, we implement a Deep Reinforcement Learning algorithm using both Keras on top of Tensorflow and PyTorch (both versions are available, you can choose the one you prefer)'},
 {'Label': 'Reinforcement Learning',
  'Sentence': 'This approach consists in the interaction between two components: an environment (the game itself) and an agent (Snake)'},
 {'Label': 'Reinforcement Learning',
  'Sentence':

### Створити та зберегти у .csv файл pandas DataFrame з обраних речень із указанням в окремому ствопчику назви напрямку (ця назва має бути однаковою для усіх речень)


In [24]:
sent_df = pd.DataFrame(data=sent_df, columns=['Sentence', 'Label'])

sent_df.to_csv(folder+'sentences.csv')

sent_df

,Sentence,Label
0,Training a virtual agent to outperform human p...,Reinforcement Learning
1,This is what Google DeepMind did with its popu...,Reinforcement Learning
2,"To do it, we implement a Deep Reinforcement Le...",Reinforcement Learning
3,This approach consists in the interaction betw...,Reinforcement Learning
4,The agent collects information about its curre...,Reinforcement Learning
...,...,...
111,The researchers also acknowledge that learning...,Reinforcement Learning
112,In “Chip Placement with Deep Reinforcement Lea...,Reinforcement Learning
113,"Unlike prior methods, our approach has the abi...",Reinforcement Learning
114,"In particular, as we train over a greater numb...",Reinforcement Learning


### Здійснити підготовку набору даних до подальшого моделювання (tokenization&embedding, train_test_split)


### Train/Test Split

In [46]:
from sklearn.model_selection import train_test_split
train_split, test_split = train_test_split(sent_df, train_size=0.8, test_size=0.2)
train_split

,Sentence,Label
38,A state is the representation of a situation i...,Reinforcement Learning
74,Learning the agent how to run is a first step ...,Reinforcement Learning
90,"In their study, the researchers at MIT Lincoln...",Reinforcement Learning
64,It’s up to the model to figure out how to perf...,Reinforcement Learning
109,And AI researchers still haven’t figured out h...,Reinforcement Learning
...,...,...
67,in usual circumstances we would require an aut...,Reinforcement Learning
47,"In our case, it consists of 3 hidden layers of...",Reinforcement Learning
49,Different architectures and different hyper-pa...,Reinforcement Learning
28,"In the example, we might want to choose RIGHT ...",Reinforcement Learning


#### Vectorization

In [32]:
max_tokens = 10000

tokens_count = 0
for sent in sentences:
  tokens_count+=len(sent.split())
avg_tokens = round(tokens_count/len(sentences))
avg_tokens

22

In [33]:
text_vectorizer = TextVectorization(max_tokens=max_tokens, # how many words in the vocabulary (all of the different words in your text)
                                    standardize="lower_and_strip_punctuation", # how to process text
                                    split="whitespace", # how to split tokens
                                    ngrams=None, # create groups of n-words?
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=avg_tokens, # how long should the output sequence of tokens be?
                                    pad_to_max_tokens=True)
text_vectorizer

In [35]:
text_vectorizer(sent_df['Sentence'])

<tf.Tensor: shape=(116, 22), dtype=int64, numpy=
array([[ 81,   4, 185, ...,   5, 107,   6],
       [ 28,  10,  34, ..., 436,   4,  45],
       [  3, 106,  19, ..., 299, 338,  38],
       ...,
       [349, 501, 564, ...,   0,   0,   0],
       [  7, 528,  23, ..., 538, 138,  16],
       [328, 686, 787, ..., 138,   7, 351]])>

In [39]:
print(f"Most Used: {text_vectorizer.get_vocabulary()[:5]}")
print(f"Most Unused: {text_vectorizer.get_vocabulary()[-5:]}")

Most Used: ['', '[UNK]', 'the', 'to', 'a']
Most Unused: ['20', '2', '120', '00005', '0000005']


#### Embedding

In [48]:
embedding = layers.Embedding(input_dim=max_tokens, # set input shape
                             output_dim=128, # set size of embedding vector
                             embeddings_initializer="uniform", # default, intialize randomly
                             input_length=avg_tokens) # how long is each input

embedding(text_vectorizer(sent_df['Sentence']))


<tf.Tensor: shape=(116, 22, 128), dtype=float32, numpy=
array([[[ 8.56719911e-04,  2.94237211e-03,  3.09188291e-03, ...,
          3.19134034e-02, -3.58311906e-02, -3.45857739e-02],
        [ 9.90051031e-03,  4.34108116e-02, -1.88305620e-02, ...,
          4.70611714e-02, -1.95220709e-02,  3.78351323e-02],
        [ 4.95757498e-02,  3.76892425e-02,  4.54409607e-02, ...,
          2.82063968e-02, -3.40144411e-02, -1.66114457e-02],
        ...,
        [ 4.33415286e-02, -4.69392426e-02, -1.49884000e-02, ...,
         -1.36009082e-02, -4.70961928e-02, -2.70201098e-02],
        [-1.51169077e-02,  1.48225911e-02, -2.98777968e-03, ...,
          2.90144347e-02, -1.01806410e-02, -1.45591274e-02],
        [ 4.00232710e-02,  4.59876768e-02, -4.04194109e-02, ...,
         -2.90425178e-02,  5.00645489e-03,  3.29445712e-02]],

       [[-4.61527221e-02,  3.96432616e-02,  4.16095518e-02, ...,
         -4.26623970e-03, -4.65283543e-03,  1.15801096e-02],
        [-1.79443732e-02, -1.85789950e-02,  3.8

### Відповіді оформити .ipynb, .csv, .pdf документами